## Data Modelleren opdracht

In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import janitor

# autoreload changes from local files
%load_ext autoreload
%autoreload 2

# pandas show full output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from datafundament.model import preprocess
from datafundament.model import normalize

## Load data

In [3]:
df_raw = pd.read_excel('data/raw/dummydataset_bestellingen.xlsx')
df_raw.head()

,Order ID,ProductNaam,Korting,Verkoop,Winst,Aantal,Categorie,Sub-Categorie,Besteldatum,KlantNaam,Stad,Land,Regio,Segment,VerzendDatum,VerstuurMethode,Provincie,Lengtegraad,breedtegraad
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45,-26,3,Office Supplies,Paper,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,18.068581,59.329324
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854,290,7,Furniture,Bookcases,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,-3.010113,53.645708
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140,21,3,Office Supplies,Art,2011-01-04,Devin Huddleston,Valence,France,Central,Consumer,2011-01-08,Economy,Auvergne-Rhône-Alpes,4.892360,44.933393
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27,-22,2,Office Supplies,Art,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,-1.890401,52.486243
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17,-1,2,Office Supplies,Storage,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,-1.890401,52.486243


Wat valt op:
- Order ID: niet uniek, ID's komen vaker voor

Opschonen:
- Kolommnamen: naar snakecase voor consistentie
- Aankoopprijs toevoegen: Verkoopprijs - winst

Vragen:
- Wat betekent een dubbele order ID - mogelijk iedere rij is een product dat gekoppeld wordt aan een order
- Visualiseer verhoudingen: producten, categoriën, landen

Data model:
- order: klant + aantal producten per order
- klant: klant informatie en locatie
- product: productnaam en categorie
- 

## Clean data

In [4]:
df = preprocess.clean_data(df_raw)
df = preprocess.add_features(df)
df.head()

,order_id,productnaam,korting,verkoop,winst,aantal,categorie,sub_categorie,besteldatum,klantnaam,stad,land,regio,segment,verzenddatum,verstuurmethode,provincie,lengtegraad,breedtegraad,winstmarge,aankoopprijs
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45,-26,3,Office Supplies,Paper,2011-01-01,Ruby Patel,Stockholm,Sweden,North,Home Office,2011-01-05,Economy Plus,Stockholm,18.068581,59.329324,-57.78,71
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854,290,7,Furniture,Bookcases,2011-01-03,Summer Hayward,Southport,United Kingdom,North,Consumer,2011-01-07,Economy,England,-3.010113,53.645708,33.96,564
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140,21,3,Office Supplies,Art,2011-01-04,Devin Huddleston,Valence,France,Central,Consumer,2011-01-08,Economy,Auvergne-Rhône-Alpes,4.892360,44.933393,15.00,119
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27,-22,2,Office Supplies,Art,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,-1.890401,52.486243,-81.48,49
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17,-1,2,Office Supplies,Storage,2011-01-04,Mary Parker,Birmingham,United Kingdom,North,Corporate,2011-01-09,Economy,England,-1.890401,52.486243,-5.88,18


## Normalization
lengtegraad breedtegraad: als klantandres dan onder customer, als bestelling dan onder feiten tabel

In [5]:
# create dim tables and update df_full with new id columns
df_geo, df_full = normalize.create_dim_dataframe(df, ['land', 'provincie', 'stad', 'regio'], 'geo_id')
df_customer, df_full = normalize.create_dim_dataframe(df_full, ['klantnaam', 'segment', 'lengtegraad', 'breedtegraad', 'geo_id'], 'klant_id')
df_product, df_full = normalize.create_dim_dataframe(df_full, ['productnaam', 'categorie', 'sub_categorie'], 'product_id')                                                             
df_customer.head()

,klant_id,klantnaam,segment,lengtegraad,breedtegraad,geo_id
0,1,Ruby Patel,Home Office,18.068581,59.329324,1
1,2,Summer Hayward,Consumer,-3.010113,53.645708,2
2,3,Devin Huddleston,Consumer,4.892360,44.933393,3
3,4,Mary Parker,Corporate,-1.890401,52.486243,4
4,5,Daniel Burke,Home Office,5.718034,45.142151,5


In [6]:
# create unique id of order + product combination, as orders are not unique
df_full['order_product_id'] = np.arange(len(df_full)) + 1

# create order facts table
# aankoop prijs en winstmarge zijn optioneel (zelf berekend)
df_order = df_full[['order_product_id', 'order_id', 'product_id', 'klant_id', 'korting', 'aankoopprijs', 'verkoop', 'winst', 'winstmarge', 'aantal', 'besteldatum', 'verzenddatum', 'verstuurmethode', 'winstmarge']]
df_order.head()

,order_product_id,order_id,product_id,klant_id,korting,aankoopprijs,verkoop,winst,winstmarge,aantal,besteldatum,verzenddatum,verstuurmethode,winstmarge
0,1,BN-2011-7407039,1,1,0.5,71,45,-26,-57.78,3,2011-01-01,2011-01-05,Economy Plus,-57.78
1,2,AZ-2011-9050313,2,2,0.0,564,854,290,33.96,7,2011-01-03,2011-01-07,Economy,33.96
2,3,AZ-2011-6674300,3,3,0.0,119,140,21,15.00,3,2011-01-04,2011-01-08,Economy,15.00
3,4,BN-2011-2819714,4,4,0.5,49,27,-22,-81.48,2,2011-01-04,2011-01-09,Economy,-81.48
4,5,BN-2011-2819714,5,4,0.5,18,17,-1,-5.88,2,2011-01-04,2011-01-09,Economy,-5.88


In [7]:
# check if all columns in full are covered
normalized_columns = set(df_geo.columns).union(set(df_customer.columns)).union(set(df_product.columns)).union(set(df_order.columns))
set(df_full.columns) - normalized_columns

set()

In [10]:
def save_dataframe(df, table_name):
    df.to_csv(f"data/processed/{table_name}.csv", index=False)

save_dataframe(df_geo, 'dim_geo')
save_dataframe(df_customer, 'dim_customer')
save_dataframe(df_product, 'dim_product')
save_dataframe(df_order, 'fact_order')